<a href="https://colab.research.google.com/github/choeuneheol/python-practice/blob/master/%EB%AC%B8%EC%A0%9C%ED%95%B4%EA%B2%B0%EC%B1%85DAY11(%EC%9A%B4%EC%A0%84%EC%9E%90%EC%98%88%EC%B8%A1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
!pip install scikit-learn
!pip install xgboost
!pip install lightgbm

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
submission = pd.read_csv('./sample_submission.csv')

In [ ]:
all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.drop('target',axis=1)

In [ ]:
all_features = all_data.columns
all_features

Index(['id', 'ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat',
       'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat',
       'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11',
       'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01',
       'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
       'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11',
       'ps_calc_12', 'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin',
       'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin',
       'ps_calc_20_bin'],
      dtyp

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_features = [feature for feature in all_features if 'cat' in feature]

onehot_encoder = OneHotEncoder()

encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

encoded_cat_matrix

<1488028x184 sparse matrix of type '<class 'numpy.float64'>'
	with 20832392 stored elements in Compressed Sparse Row format>

In [ ]:
drop_features = ['ps_ind_14','ps_ind_10_bin','ps_ind_11_bin',
                 'ps_ind_12_bin','ps_ind_13_bin','ps_car_14']

remaining_features = [feature for feature in all_features
                      if('cat' not in feature and
                         'calc'not in feature and
                         feature not in drop_features)]

In [ ]:
from scipy import sparse

all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data[remaining_features]),
                               encoded_cat_matrix],
                              format='csr')

In [ ]:
num_train = len(train)

x = all_data_sprs[:num_train]
x_test = all_data_sprs[num_train:]

y=train['target'].values

In [ ]:
import numpy as np

def eval_gini(y_true, y_pred):
  assert y_true.shape == y_pred.shape

  n_samples = y_true.shape[0]
  L_mid = np.linspace(1/n_samples, 1, n_samples)

  pred_order = y_true[y_pred.argsort()]
  L_pred = np.cumsum(pred_order) / np.sum(pred_order)
  G_pred = np.sum(L_mid - L_pred)

  true_order = y_true[y_true.argsort()]
  L_true = np.cumsum(true_order) / np.sum(true_order)
  G_true = np.sum(L_mid - L_true)

  return G_pred / G_true

In [ ]:
def gini(preds, dtrain):
  labels = dtrain.get_label()
  return 'gini', eval_gini(labels, preds), True

In [ ]:
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

In [ ]:
params = {'objective':'binary',
          'learning_rate':0.01,
          'force_row_wise': True,
          'random_state':0}

In [ ]:
oof_val_preds = np.zeros(x.shape[0])

oof_test_preds = np.zeros(x_test.shape[0])

In [ ]:
pip install lightgbm

In [ ]:
import lightgbm as lgb

for idx, (train_idx, valid_idx) in enumerate(folds.split(x,y)):

  print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}','#'*40)

  x_train, y_train = x[train_idx], y[train_idx]
  x_valid, y_valid = x[valid_idx], y[valid_idx]

  dtrain = lgb.Dataset(x_train, y_train)
  dvalid = lgb.Dataset(x_valid, y_valid)

  lgb_model = lgb.train(params=params,
                        train_set=dtrain,
                        num_boost_round=1000,
                        valid_sets=dvalid,
                        feval = gini,
                        early_stopping_rounds=100,
                        verbose_eval=100)
  
  oof_test_preds += lgb_model.predict(x_test)/folds.n_splits

  oof_val_preds[valid_idx] += lgb_model.predict(x_valid)

  gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])

  print(f'폴드 {idx+1} 지니계수: {gini_score}\n')

######################################## 폴드 1 / 폴드 5 ########################################


C:\Users\easy1\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\easy1\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1350
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 201
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[LightGBM] [Info] Start training from score -3.274764
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.15335	valid_0's gini: 0.261928
[200]	valid_0's binary_logloss: 0.152433	valid_0's gini: 0.275632
[300]	valid_0's binary_logloss: 0.152028	valid_0's gini: 0.282552
[400]	valid_0's binary_logloss: 0.151815	valid_0's gini: 0.286654
[500]	valid_0's binary_logloss: 0.151734	valid_0's gini: 0.288179
[600]	valid_0's binary_logloss: 0.151684	valid_0's gini: 0.289048
[700]	valid_0's binary_logloss: 0.151664	valid_0's gini: 0.289671
[800]	valid_0's binary_logloss: 0.151656	valid_0's gini: 0.289967
[900]	valid_0's binary_logloss: 0.151649	valid_0's gini: 0.290064
Early stop

In [ ]:
print('OOF 검증 데이터 지니계수:', eval_gini(y, oof_val_preds))

OOF 검증 데이터 지니계수: 0.2802369580498456


In [ ]:
submission['target'] = oof_test_preds
submission.to_csv('submission.csv')

In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('submission.csv')

In [ ]:
all_data = pd.concat([train, test], ignore_index=True)
all_data = all_data.drop('target', axis=1)

all_features = all_data.columns

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_features = [feature for feature in all_features if 'cat' in feature]

onehot_encoder = OneHotEncoder()
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

In [ ]:
all_data['num_missing'] = (all_data==-1).sum(axis=1)

In [ ]:
remaining_features = [feature for feature in all_features
                      if ('cat' not in feature and 'calc' not in feature)]
remaining_features.append('num_missing')

In [ ]:
ind_features = [feature for feature in all_features if 'ind' in feature]

is_first_feature = True
for ind_feature in ind_features:
  if is_first_feature:
    all_data['mix_ind'] = all_data[ind_feature].astype(str) + '_'
    is_first_feature = False
  else:
    all_data['mix_ind'] += all_data[ind_feature].astype(str) + '_'

In [ ]:
all_data['mix_ind']

0          2_2_5_1_0_0_1_0_0_0_0_0_0_0_11_0_1_0_
1           1_1_7_0_0_0_0_1_0_0_0_0_0_0_3_0_0_1_
2          5_4_9_1_0_0_0_1_0_0_0_0_0_0_12_1_0_0_
3           0_1_2_0_0_1_0_0_0_0_0_0_0_0_8_1_0_0_
4           0_2_0_1_0_1_0_0_0_0_0_0_0_0_9_1_0_0_
                           ...                  
1488023     0_1_6_0_0_0_1_0_0_0_0_0_0_0_2_0_0_1_
1488024    5_3_5_1_0_0_0_1_0_0_0_0_0_0_11_1_0_0_
1488025     0_1_5_0_0_1_0_0_0_0_0_0_0_0_5_0_0_1_
1488026    6_1_5_1_0_0_0_0_1_0_0_0_0_0_13_1_0_0_
1488027    7_1_4_1_0_0_0_0_1_0_0_0_0_0_12_1_0_0_
Name: mix_ind, Length: 1488028, dtype: object

In [ ]:
all_data['ps_ind_02_cat'].value_counts()

 1    1079327
 2     309747
 3      70172
 4      28259
-1        523
Name: ps_ind_02_cat, dtype: int64

In [ ]:
all_data['ps_ind_02_cat'].value_counts().to_dict()

{1: 1079327, 2: 309747, 3: 70172, 4: 28259, -1: 523}

In [ ]:
cat_count_features = []
for feature in cat_features+['mix_ind']:
  val_counts_dict = all_data[feature].value_counts().to_dict()
  all_data[f'{feature}_count'] = all_data[feature].apply(lambda x:
                                                         val_counts_dict[x])
  cat_count_features.append(f'{feature}_count')

In [ ]:
cat_count_features

['ps_ind_02_cat_count',
 'ps_ind_04_cat_count',
 'ps_ind_05_cat_count',
 'ps_car_01_cat_count',
 'ps_car_02_cat_count',
 'ps_car_03_cat_count',
 'ps_car_04_cat_count',
 'ps_car_05_cat_count',
 'ps_car_06_cat_count',
 'ps_car_07_cat_count',
 'ps_car_08_cat_count',
 'ps_car_09_cat_count',
 'ps_car_10_cat_count',
 'ps_car_11_cat_count',
 'mix_ind_count']

In [ ]:
from scipy import sparse
!pip3 install bayesian-optimization
drop_features = ['ps_ind_14','ps_ind_10_bin','ps_ind_11_bin',
                 'ps_ind_12_bin','ps_ind_13_bin','ps_car_14']

all_data_remaining = all_data[remaining_features+cat_count_features].drop(drop_features, axis=1)
all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data_remaining),
                               encoded_cat_matrix],
                              format='csr')

In [ ]:
num_train = len(train)

x = all_data_sprs[:num_train]
x_test = all_data_sprs[num_train:]

y = train['target'].values

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x, y,
                                                      test_size=0.2,
                                                      random_state=0)

bayes_dtrain = lgb.Dataset(x_train, y_train)
bayes_dvalid = lgb.Dataset(x_valid, y_valid)

In [ ]:
param_bounds = {'num_leaves' : (30, 40),
                'lambda_l1' : (0.7, 0.9),
                'lambda_l2' : (0.9, 1),
                'feature_fraction' : (0.6, 0.7),
                'bagging_fraction' : (0.6, 0.9),
                'min_child_samples' : (6, 10),
                'min_child_weight' : (10, 40)}

fixed_params = {'objective' : 'binary',
                'learning_rate' : 0.005,
                'bagging_freq' : 1,
                'force_row_wise' : True,
                'random_state' : 1991}

In [ ]:
def eval_function(num_leaves, lambda_l1, lambda_l2, feature_fraction,
                  bagging_fraction, min_child_samples, min_child_weight):
  '''최적화하려는 평가지표(지니계수) 계산 함수'''

  params = {'num_leaves' : int(round(num_leaves)),
            'lambda_l1': lambda_l1,
            'lambda_l2': lambda_l2,
            'feature_fraction': feature_fraction,
            'bagging_fraction': bagging_fraction,
            'min_child_samples': int(round(min_child_samples)),
            'min_child_weight': min_child_weight,
            'feature_pre_filter': False}
  params.update(fixed_params)

  print('하이퍼파라미터:', params)

  lgb_model = lgb.train(params=params,
                        train_set=bayes_dtrain,
                        num_boost_round=2500,
                        valid_sets=bayes_dvalid,
                        feval=gini,
                        early_stopping_rounds=300,
                        verbose_eval=False)
  preds = lgb_model.predict(x_valid)

  gini_score = eval_gini(y_valid, preds)
  print(f'지니계수 : {gini_score}\n')

  return gini_score

In [ ]:
from bayes_opt import BayesianOptimization

optimizer = BayesianOptimization(f=eval_function,
                                 pbounds=param_bounds,
                                 random_state=0)

In [ ]:
optimizer.maximize(init_points=3, n_iter=6)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | min_ch... | min_ch... | num_le... |
-------------------------------------------------------------------------------------------------------------
하이퍼파라미터: {'num_leaves': 34, 'lambda_l1': 0.8205526752143287, 'lambda_l2': 0.9544883182996897, 'feature_fraction': 0.6715189366372419, 'bagging_fraction': 0.7646440511781974, 'min_child_samples': 8, 'min_child_weight': 29.376823391999682, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}


C:\Users\easy1\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\easy1\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 17383, number of negative: 458786
[LightGBM] [Info] Total Bins 1810
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 218
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036506 -> initscore=-3.273091
[LightGBM] [Info] Start training from score -3.273091
지니계수 : 0.28363010190508786

|  1        |  0.2836   |  0.7646   |  0.6715   |  0.8206   |  0.9545   |  7.695    |  29.38    |  34.38    |
하이퍼파라미터: {'num_leaves': 39, 'lambda_l1': 0.7766883037651555, 'lambda_l2': 0.9791725038082665, 'feature_fraction': 0.6963662760501029, 'bagging_fraction': 0.867531900234624, 'min_child_samples': 8, 'min_child_weight': 27.04133683281797, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}
[LightGBM] [Info] Number of positive: 17383, number of negative: 458786
[LightGBM] [Info] Total Bins 1810
[LightGBM] [Info] Number of data points in t

C:\Users\easy1\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\easy1\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Total Bins 1810
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 218
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036506 -> initscore=-3.273091
[LightGBM] [Info] Start training from score -3.273091
지니계수 : 0.28436117538603967

|  4        |  0.2844   |  0.6      |  0.6073   |  0.7069   |  0.9993   |  9.503    |  38.8     |  39.42    |
하이퍼파라미터: {'num_leaves': 36, 'lambda_l1': 0.9, 'lambda_l2': 0.9, 'feature_fraction': 0.6, 'bagging_fraction': 0.9, 'min_child_samples': 6, 'min_child_weight': 36.88499584533189, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}
[LightGBM] [Info] Number of positive: 17383, number of negative: 458786
[LightGBM] [Info] Total Bins 1810


...

지니계수 : 0.2833493148249361

|  7        |  0.2833   |  0.8674   |  0.6875   |  0.7      |  1.0      |  7.983    |  37.27    |  40.0     |
하이퍼파라미터: {'num_leaves': 32, 'lambda_l1': 0.829932662510154, 'lambda_l2': 0.9309382272206734, 'feature_fraction': 0.6952194054160209, 'bagging_fraction': 0.6675659367709144, 'min_child_samples': 8, 'min_child_weight': 28.604068940720285, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}
[LightGBM] [Info] Number of positive: 17383, number of negative: 458786
[LightGBM] [Info] Total Bins 1810
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 218
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036506 -> initscore=-3.273091
[LightGBM] [Info] Start training from score -3.273091


C:\Users\easy1\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\easy1\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


지니계수 : 0.28406393088341925

|  8        |  0.2841   |  0.6676   |  0.6952   |  0.8299   |  0.9309   |  7.88     |  28.6     |  31.94    |
하이퍼파라미터: {'num_leaves': 40, 'lambda_l1': 0.7455553867388193, 'lambda_l2': 0.9832428375902124, 'feature_fraction': 0.6593794164233446, 'bagging_fraction': 0.7795604634116916, 'min_child_samples': 9, 'min_child_weight': 36.205421876982925, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 1991}
[LightGBM] [Info] Number of positive: 17383, number of negative: 458786
[LightGBM] [Info] Total Bins 1810
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 218
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036506 -> initscore=-3.273091
[LightGBM] [Info] Start training from score -3.273091


C:\Users\easy1\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\easy1\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


지니계수 : 0.28332196800720755

|  9        |  0.2833   |  0.7796   |  0.6594   |  0.7456   |  0.9832   |  9.232    |  36.21    |  39.69    |


In [ ]:
max_params=optimizer.max['params']
max_params

{'bagging_fraction': 0.6213108174593661,
 'feature_fraction': 0.608712929970154,
 'lambda_l1': 0.7040436794880651,
 'lambda_l2': 0.9832619845547939,
 'min_child_samples': 9.112627003799401,
 'min_child_weight': 36.10036444740457,
 'num_leaves': 39.78618342232764}

In [ ]:
max_params['num_leaves']=int(round(max_params['num_leaves']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))

In [ ]:
max_params.update(fixed_params)

In [ ]:
max_params

{'bagging_fraction': 0.6213108174593661,
 'feature_fraction': 0.608712929970154,
 'lambda_l1': 0.7040436794880651,
 'lambda_l2': 0.9832619845547939,
 'min_child_samples': 9,
 'min_child_weight': 36.10036444740457,
 'num_leaves': 40,
 'objective': 'binary',
 'learning_rate': 0.005,
 'bagging_freq': 1,
 'force_row_wise': True,
 'random_state': 1991}

In [ ]:
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

oof_val_preds = np.zeros(x.shape[0])

oof_test_preds = np.zeros(x_test.shape[0])

for idx, (train_idx, valid_idx) in enumerate(folds.split(x,y)):
  print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}','#'*40)

  x_train, y_train = x[train_idx], y[train_idx]
  x_valid, y_valid = x[valid_idx], y[valid_idx]

  dtrain = lgb.Dataset(x_train, y_train)
  dvalid = lgb.Dataset(x_valid, y_valid)

  lgb_model = lgb.train(params=max_params,
                        train_set=dtrain,
                        num_boost_round=2500,
                        valid_sets=dvalid,
                        feval=gini,
                        early_stopping_rounds=300,
                        verbose_eval=100)
  
  oof_test_preds += lgb_model.predict(x_test)/folds.n_splits

  oof_val_preds[valid_idx] += lgb_model.predict(x_valid)

  gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
  print(f'폴드 {idx+1} 지니계수 : {gini_score}\n')

######################################## 폴드 1 / 폴드 5 ########################################


C:\Users\easy1\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\easy1\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1809
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 217
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[LightGBM] [Info] Start training from score -3.274764
Training until validation scores don't improve for 300 rounds
[100]	valid_0's binary_logloss: 0.154244	valid_0's gini: 0.270164
[200]	valid_0's binary_logloss: 0.153169	valid_0's gini: 0.275505
[300]	valid_0's binary_logloss: 0.152586	valid_0's gini: 0.279508
[400]	valid_0's binary_logloss: 0.152231	valid_0's gini: 0.282936
[500]	valid_0's binary_logloss: 0.152003	valid_0's gini: 0.285883
[600]	valid_0's binary_logloss: 0.151835	valid_0's gini: 0.288718
[700]	valid_0's binary_logloss: 0.15172	valid_0's gini: 0.290805
[800]	valid_0's binary_logloss: 0.151631	valid_0's gini: 0.292574
[900]	valid_0's binary_logloss: 0.151564	valid_0's gini: 0.294074
[1000]	val

[100]	valid_0's binary_logloss: 0.154403	valid_0's gini: 0.267477
[200]	valid_0's binary_logloss: 0.153386	valid_0's gini: 0.272538
[300]	valid_0's binary_logloss: 0.152834	valid_0's gini: 0.276025
[400]	valid_0's binary_logloss: 0.152495	valid_0's gini: 0.279599
[500]	valid_0's binary_logloss: 0.152273	valid_0's gini: 0.28273
[600]	valid_0's binary_logloss: 0.152133	valid_0's gini: 0.284935
[700]	valid_0's binary_logloss: 0.152022	valid_0's gini: 0.287113
[800]	valid_0's binary_logloss: 0.151943	valid_0's gini: 0.288886
[900]	valid_0's binary_logloss: 0.151893	valid_0's gini: 0.289989
[1000]	valid_0's binary_logloss: 0.151846	valid_0's gini: 0.29118
[1100]	valid_0's binary_logloss: 0.151806	valid_0's gini: 0.292204
[1200]	valid_0's binary_logloss: 0.151783	valid_0's gini: 0.292725
[1300]	valid_0's binary_logloss: 0.15176	valid_0's gini: 0.293274
[1400]	valid_0's binary_logloss: 0.151746	valid_0's gini: 0.293713
[1500]	valid_0's binary_logloss: 0.151735	valid_0's gini: 0.294002
[1600]	

In [ ]:
print('OOF 검증 데이터 지니계수 :', eval_gini(y, oof_val_preds))

OOF 검증 데이터 지니계수 : 0.288773243692427


In [ ]:
submission['target'] = oof_test_preds
submission.to_csv('submission.csv')

In [ ]:
def gini(preds, dtrain):
  labels = dtrain.get_label()
  return 'gini', eval_gini(labels, preds), True

In [ ]:
def gini(preds, dtrain):
  labels = dtrain.get_label()
  return 'gini', eval_gini(labels,preds)

In [ ]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x,y,
                                                      test_size=0.2,
                                                      random_state=0)

bayes_dtrain = xgb.DMatrix(x_train, y_train)
bayes_dvalid = xgb.DMatrix(x_valid, y_valid)

In [ ]:
param_bounds = {'max_depth':(4, 8),
                'subsample':(0.6,0.9),
                'colsample_bytree':(0.7, 1.0),
                'min_child_weight':(5, 7),
                'gamma':(8,11),
                'reg_alpha':(7,9),
                'reg_lambda':(1.1, 1.5),
                'scale_pos_weight':(1.4, 1.6)}

fixed_params = {'objective':'binary:logistic',
                'learning_rete':0.02,
                'random_state':1991}

In [ ]:
def eval_function(max_depth, subsample, colsample_bytree, min_child_weight,
                  reg_alpha, gamma, reg_lambda, scale_pos_weight):
  '''최적화하려는 평가지표(지니계수) 계산 함수'''
  params = {'max_depth':int(round(max_depth)),
            'subsample':subsample,
            'colsample_bytree':colsample_bytree,
            'min_child_weight':min_child_weight,
            'gamma':gamma,
            'reg_alpha':reg_alpha,
            'reg_lambda':reg_lambda,
            'scale_pos_weight':scale_pos_weight}

  params.update(fixed_params)

  print('하이퍼파라미터 :', params)

  xgb_model = xgb.train(params=params,
                        dtrain=bayes_dtrain,
                        num_boost_round=2000,
                        evals=[(bayes_dvalid, 'bayes_dvalid')],
                        maximize=True,
                        feval=gini,
                        early_stopping_rounds=200,
                        verbose_eval=False)
  
  best_iter = xgb_model.best_iteration

  preds = xgb_model.predict(bayes_dvalid,
                            iteration_range=(0, best_iter))
  
  gini_score = eval_gini(y_valid, preds)
  print(f'지니계수 : {gini_score}\n')

  return gini_score

In [ ]:
from bayes_opt import BayesianOptimization

optimizer = BayesianOptimization(f=eval_function,
                                 pbounds=param_bounds,
                                 random_state=0)

optimizer.maximize(init_points=3, n_iter=6)

|   iter    |  target   | colsam... |   gamma   | max_depth | min_ch... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------
하이퍼파라미터 : {'max_depth': 6, 'subsample': 0.867531900234624, 'colsample_bytree': 0.8646440511781974, 'min_child_weight': 6.0897663659937935, 'gamma': 10.14556809911726, 'reg_alpha': 7.84730959867781, 'reg_lambda': 1.3583576452266626, 'scale_pos_weight': 1.4875174422525386, 'objective': 'binary:logistic', 'learning_rete': 0.02, 'random_state': 1991}
[14:36:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "learning_rete" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any

C:\Users\easy1\anaconda3\lib\site-packages\xgboost\training.py:35: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


지니계수 : 0.272454262578766

|  1        |  0.2725   |  0.8646   |  10.15    |  6.411    |  6.09     |  7.847    |  1.358    |  1.488    |  0.8675   |
하이퍼파라미터 : {'max_depth': 7, 'subsample': 0.6261387899104622, 'colsample_bytree': 0.9890988281503088, 'min_child_weight': 6.0577898395058085, 'gamma': 9.150324556477333, 'reg_alpha': 8.136089122187865, 'reg_lambda': 1.4702386553170643, 'scale_pos_weight': 1.4142072116395774, 'objective': 'binary:logistic', 'learning_rete': 0.02, 'random_state': 1991}
[14:37:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "learning_rete" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


지니계수 : 0.27126092557530607

|  2        |  0.2713   |  0.9891   |  9.15     |  

C:\Users\easy1\anaconda3\lib\site-packages\xgboost\training.py:35: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


지니계수 : 0.27566003503708425

|  4        |  0.2757   |  0.7938   |  10.79    |  7.001    |  6.662    |  8.924    |  1.452    |  1.573    |  0.8665   |
하이퍼파라미터 : {'max_depth': 6, 'subsample': 0.9, 'colsample_bytree': 1.0, 'min_child_weight': 7.0, 'gamma': 11.0, 'reg_alpha': 9.0, 'reg_lambda': 1.5, 'scale_pos_weight': 1.6, 'objective': 'binary:logistic', 'learning_rete': 0.02, 'random_state': 1991}
[14:39:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "learning_rete" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




...

지니계수 : 0.2750306700444254

|  7        |  0.275    |  0.9054   |  11.0     |  7.406    |  6.991    |  8.741    |  1.15     |  1.545    |  0.8718   |
하이퍼파라미터 : {'max_depth': 4, 'subsample': 0.9, 'colsample_bytree': 0.7, 'min_child_weight': 5.0, 'gamma': 8.0, 'reg_alpha': 7.0, 'reg_lambda': 1.1, 'scale_pos_weight': 1.6, 'objective': 'binary:logistic', 'learning_rete': 0.02, 'random_state': 1991}
[14:42:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "learning_rete" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\Users\easy1\anaconda3\lib\site-packages\xgboost\training.py:35: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


지니계수 : 0.27727823233016896

|  8        |  0.2773   |  0.7      |  8.0      |  4.0      |  5.0      |  7.0      |  1.1      |  1.6      |  0.9      |
하이퍼파라미터 : {'max_depth': 4, 'subsample': 0.7115165552982349, 'colsample_bytree': 0.7, 'min_child_weight': 5.0, 'gamma': 8.0, 'reg_alpha': 8.01124778269624, 'reg_lambda': 1.1, 'scale_pos_weight': 1.6, 'objective': 'binary:logistic', 'learning_rete': 0.02, 'random_state': 1991}


C:\Users\easy1\anaconda3\lib\site-packages\xgboost\training.py:35: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[14:43:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "learning_rete" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


지니계수 : 0.2772801651675007

|  9        |  0.2773   |  0.7      |  8.0      |  4.0      |  5.0      |  8.011    |  1.1      |  1.6      |  0.7115   |


In [ ]:
max_params = optimizer.max['params']
max_params

{'colsample_bytree': 0.7,
 'gamma': 8.0,
 'max_depth': 4.0,
 'min_child_weight': 5.0,
 'reg_alpha': 8.01124778269624,
 'reg_lambda': 1.1,
 'scale_pos_weight': 1.6,
 'subsample': 0.7115165552982349}

In [ ]:
max_params['max_depth'] = int(round(max_params['max_depth']))
max_params.update(fixed_params)
max_params

{'colsample_bytree': 0.7,
 'gamma': 8.0,
 'max_depth': 4,
 'min_child_weight': 5.0,
 'reg_alpha': 8.01124778269624,
 'reg_lambda': 1.1,
 'scale_pos_weight': 1.6,
 'subsample': 0.7115165552982349,
 'objective': 'binary:logistic',
 'learning_rete': 0.02,
 'random_state': 1991}

In [ ]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install -U scikit-learn

In [ ]:
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

oof_val_preds = np.zeros(x.shape[0])

In [ ]:
oof_test_preds = np.zeros(x_test.shape[0])

for idx, (train_idx, valid_idx) in enumerate(folds.split(x,y)):
  print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}','#'*40)

  x_train, y_train = x[train_idx], y[train_idx]
  x_valid, y_valid = x[valid_idx], y[valid_idx]

  dtrain = xgb.DMatrix(x_train, y_train)
  dvalid = xgb.DMatrix(x_valid, y_valid)  
  dtest = xgb.DMatrix(x_test)

  xgb_model = xgb.train(params=max_params,
                        dtrain=dtrain,
                        num_boost_round=2000,
                        evals=[(dvalid, 'valid')],
                        maximize=True,
                        feval=gini,
                        early_stopping_rounds=200,
                        verbose_eval=100)
  
  best_iter = xgb_model.best_iteration

  oof_test_preds += xgb_model.predict(dtest,
                                     iteration_range=(0, best_iter))/folds.n_splits

  oof_val_preds[valid_idx] += xgb_model.predict(dvalid,
                                                iteration_range=(0, best_iter))
  
  gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
  print(f'폴드 {idx+1} 지니계수 : {gini_score}\n')

######################################## 폴드 1 / 폴드 5 ########################################


C:\Users\easy1\anaconda3\lib\site-packages\xgboost\training.py:35: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(


[16:11:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "learning_rete" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	valid-logloss:0.48153	valid-gini:0.18146
[100]	valid-logloss:0.15612	valid-gini:0.28761
[200]	valid-logloss:0.15629	valid-gini:0.28306
[264]	valid-logloss:0.15651	valid-gini:0.27795
폴드 1 지니계수 : 0.29055187743212463

######################################## 폴드 2 / 폴드 5 ########################################
[16:11:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "learning_rete" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistaken

In [ ]:
print('OOF 검증 데이터 지니계수 :', eval_gini(y, oof_val_preds))

OOF 검증 데이터 지니계수 : 0.2797635481815385


In [ ]:
submission['target'] = oof_test_preds
submission.to_csv('submission.csv')

In [ ]:
oof_test_preds = oof_val_preds * 0.5 + oof_val_preds * 0.5

In [ ]:
print(oof_test_preds)

[0.08002885 0.05348538 0.02769019 ... 0.02414416 0.04554787 0.04266113]


ValueError: Length of values (595212) does not match length of index (892816)